# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
data = pd.read_csv("weather_data.csv")
data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sinjar,36.32,41.88,84.09,16,0,6.29,IQ,1594862715
1,mataura,-46.19,168.86,44.60,75,46,3.38,NZ,1594862715
2,anadyr,64.75,177.48,59.00,77,20,13.42,RU,1594862715
3,atambua,-9.11,124.89,78.76,56,97,6.24,ID,1594862715
4,sterling,39.01,-77.43,82.99,51,75,9.17,US,1594862715
...,...,...,...,...,...,...,...,...,...
550,moussoro,13.64,16.49,85.28,42,98,5.26,TD,1594862778
551,lashio,22.93,97.75,72.16,98,100,0.60,MM,1594862778
552,tingi,-11.30,35.03,49.69,78,62,3.27,TZ,1594862778
553,wakkanai,45.41,141.67,68.00,77,75,2.24,JP,1594862778


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
#Declaring Lat and Lng as locations
locations = data[["Lat", "Lng"]]
fig = gmaps.figure()
Humidity = data["Humidity"].astype(float)

In [30]:
#Configuring gmaps using Humidty as weight
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
weather = pd.DataFrame(data)
perfect_weather = weather.loc[(weather["Max Temp"]>70)&(weather["Max Temp"]<85)&(weather["Wind Speed"]<10)&(weather["Cloudiness"]==1)].dropna()
perfect_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,jamestown,42.10,-79.24,77.00,57,1,2.93,US,1594862594
22,shelburne,44.08,-80.20,73.99,60,1,6.93,CA,1594862717
34,mayo,38.89,-76.51,80.01,83,1,9.17,US,1594862719
163,pullman,46.73,-117.18,84.00,26,1,5.82,US,1594862732
354,broome,42.25,-75.83,75.00,56,1,4.70,US,1594862753
360,saint anthony,45.02,-93.22,80.01,31,1,6.93,US,1594862754
367,anchorage,61.22,-149.90,78.80,34,1,9.17,US,1594862720
457,huntsville,34.73,-86.59,84.20,62,1,4.16,US,1594862581
515,wadena,46.44,-95.14,75.20,56,1,3.36,US,1594862773


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df=perfect_weather[["City","Lat","Lng","Country"]].copy()
hotel_df["Hotel Name"]=""
hotel_df

,City,Lat,Lng,Country,Hotel Name
10,jamestown,42.10,-79.24,US,
22,shelburne,44.08,-80.20,CA,
34,mayo,38.89,-76.51,US,
163,pullman,46.73,-117.18,US,
354,broome,42.25,-75.83,US,
360,saint anthony,45.02,-93.22,US,
367,anchorage,61.22,-149.90,US,
457,huntsville,34.73,-86.59,US,
515,wadena,46.44,-95.14,US,


In [33]:
# geocoordinates
#target_search = "Chinese"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hoteldata=response.json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hoteldata["results"][0]["name"]
    except(KeyError,IndexError):
        print("Got to the next hotel")
print(json.dumps(hoteldata,indent=4,sort_keys=True))
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 46.4464607,
                    "lng": -95.145668
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.4475333802915,
                        "lng": -95.1445767197085
                    },
                    "southwest": {
                        "lat": 46.4448354197085,
                        "lng": -95.1472746802915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "2bca0b7bcfd521b0fcebaefd95a26361d5461d7d",
            "name": "Eastwood Inn of Wadena",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1440,
                 

,City,Lat,Lng,Country,Hotel Name
10,jamestown,42.10,-79.24,US,Hampton Inn & Suites Jamestown
22,shelburne,44.08,-80.20,CA,The Retreat at the Farm
34,mayo,38.89,-76.51,US,Resorts Resource Group International
163,pullman,46.73,-117.18,US,Holiday Inn Express & Suites Pullman
354,broome,42.25,-75.83,US,Chenango Valley State Park
360,saint anthony,45.02,-93.22,US,Holiday Inn Express Roseville-St. Paul
367,anchorage,61.22,-149.90,US,Hotel Captain Cook
457,huntsville,34.73,-86.59,US,Embassy Suites by Hilton Huntsville Hotel & Spa
515,wadena,46.44,-95.14,US,Eastwood Inn of Wadena


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map
marker=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))